#### Seq2Seq 모델 구현하기

In [ ]:
import torch
import torch.nn as nn
import random
import matplotlib.pyplot as plt

vocab_size=256  #총아스키코드의 개수

#입력될 원문과 번역문을 아스키 코드의 배열로 정의 및 파이토치의 텐서로 변환
x_=list(map(ord,'hello'))
y_=list(map(ord,'hola'))
print('hello - > ',x_)
print('halo - > ',y_)
x=torch.LongTensor(x_)
y=torch.LongTensor(y_)

In [ ]:
# Seq2Seq 모델 클래스 정의
class Seq2Seq(nn.Module):
  def __init__(self,vocab_size,hidden_size):
    super(Seq2Seq,self).__init__()
    self.n_layers=1
    self.hidden_size=hidden_size
    self.embedding=nn.Embedding(vocab_size,hidden_size)
    # 인코더, 디코더를 GRU 객체로 정의
    self.encoder=nn.GRU(hidden_size,hidden_size) #인코더
    self.decoder=nn.GRU(hidden_size,hidden_size) #디코더
    self.project=nn.Linear(hidden_size,vocab_size) # 다음 토큰을 예상해내는 작은 신경망 하나 더 만들기
  
  #forward 함수 정의
  def forward(self,inputs,targets):
    # 인코더의 첫번째 은닉벡터 정의
    initial_state=self._init_state()
    #원문('hello')을 구성하는 문자 임베딩
    embedding=self.embedding(inputs).unsqueeze(1)
    #원문을 인코더에 입력하여 문맥 벡터에 해당하는 encoder_state생성
    #생성한 encoder_state를 디코더의 첫번째 은닉벡터로 지정
    encoder_output,encoder_state=self.encoder(embedding,initial_state)
    decoder_state=encoder_state
    # decoder에서 문장의 시작 토큰을 입력받기 위해 아스키값으로 공백문자를 뜻하는 0으로 설정
    decoder_input=torch.LongTensor([0])
    outputs=[]

    # for loop: 아스키 번호 0을 이용해 번역문 토큰을 예측했으면 이 예측한 토큰을 이용해 또 다음 토큰을 예측
    for i in range(targets.size()[0]):
      # decoder는 첫 번째 토큰과 인코더의 문맥 벡터를 동시에 입력받음
      decoder_input=self.embedding(decoder_input).unsqueeze(1)
      decoder_output,decoder_state=self.decoder(decoder_input,decoder_state)

      projection=self.project(decoder_output)
      outputs.append(projection)
      #티처 포싱을 이용한 디코더 입력 갱신
      decoder_input=torch.LongTensor([targets[i]])
    outputs=torch.stack(outputs).squeeze()
    return outputs

  def _init_state(self,batch_size=1):
    weight=next(self.parameters()).data
    return weight.new(self.n_layers,batch_size,self.hidden_size).zero_()


In [ ]:
# 오차함수와 최적화 알고리즘 정의
seq2seq=Seq2Seq(vocab_size,16)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(seq2seq.parameters(),lr=1e-3)

log=[]
for i in range(1000):
  prediction=seq2seq(x,y)
  loss=criterion(prediction,y)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  loss_val=loss.data
  log.append(loss_val)
  if i%100==0:
    print('\n 반복:%d 오차:%s'%(i,loss_val.item()))
    _,top1=prediction.data.topk(1,1)
  print([chr(c) for c in top1.squeeze().numpy().tolist()])
plt.plot(log)
plt.ylabel('cross entropy loss')
plt.show()